<a href="https://colab.research.google.com/github/nicc0614/2025.02.18/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install line-bot-sdk
!pip install flask
!pip install pyngrok
!pip install flask_ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 6.0 MB/s eta 0:00:00


In [3]:
#colab專用
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

def ngrok_start():
  ngrok.set_auth_token(userdata.get('ngrok_authtoken'))
  ngrok.connect(5000)
  run_with_ngrok(app)

In [7]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)

from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,#回覆訊息
    TextMessage,#文字訊息
    StickerMessage,#貼圖訊息
    ImageMessage,#圖片訊息
    VideoMessage,#影片訊息
    LocationMessage#地圖訊息
)
#通知有人傳文字訊息
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('Line_channel_token'))
handler = WebhookHandler(userdata.get('line_channel_secret'))

#路由
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True) #line平台傳來的訊息
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError: #如果發生錯誤
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

#詢問gemini
from google.colab import userdata
import google.generativeai as genai
genai.configure(api_key=userdata.get("google_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash-exp")
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text

#如果收到MessageEvent就做下面的事
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        action = event.message.text
        if action =='sticker':
          #回覆貼圖
          reply = StickerMessage(
              package_id='446',
              sticker_id='1989'
)
        elif action =='image':
          #回覆圖片
          reply = ImageMessage(
              original_content_url='https://rpb206.nsysu.edu.tw/static/file/369/1369/randimg/mobileadv_6184_9270778_90311.jpg',
              preview_image_url='https://rpb206.nsysu.edu.tw/static/file/369/1369/randimg/mobileadv_6184_9270778_90311.jpg'
)
        elif action =='video':
          #回覆影片
          reply = VideoMessage(
              original_content_url='https://youtu.be/Vh7dVfQLdAs',
              preview_image_url='https://youtu.be/Vh7dVfQLdAs'
)
        elif action =='location':
          #回覆位置
          reply = LocationMessage(
              title="國立中山大學",
              address="804高雄市鼓山區蓮海路70號",
              latitude=22.628883060093656,
              longitude=120.26457969592747
)
        else:
          response = ask_gemini(action)
          reply = TextMessage(text=response)#回覆文字訊息

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )
#colab專用
ngrok_start()
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://fa14-34-80-141-228.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:57:04] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:57:15] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:57:20] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:57:25] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:57:44] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/May/2025 09:57:51] "POST /callback HTTP/1.1" 200 -
